# --- Day 16: Proboscidea Volcanium ---

https://adventofcode.com/2022/day/16

## Get Input Data

In [1]:
import re

def parse_input(filename):
    """Parse input data for today's puzzle.
    
    Parameters
    ----------
    filename : str
        The name of the *.txt file in the inputs/ directory.
    
    Returns
    -------
    graph : dict
    flow_rates : dict
    """
    graph = {}
    flow_rates = {}

    pattern = r"Valve ([A-Z]{2}) .+=(\d+); .+ valves? (.+)"
    file_str = open(f"../inputs/{filename}.txt").read()

    for node, flow_rate, neighbors in re.findall(pattern, file_str):
        graph[node] = {}
        for neighbor in neighbors.split(", "):
            # NB: Adding a distance of 1 betwen each node.
            # This creates a *weighted* graph, with all the weights equal to 1
            # This becomes useful later to have this structured like this because
            # I use Floyd-Warshall to find the min distance between all nodes in 
            # the graph.
            graph[node][neighbor] = 1
                
        flow_rates[node] = int(flow_rate)

    return graph, flow_rates

In [2]:
test_valves_graph, test_flow_rates = parse_input("test_valves")
test_valves_graph

{'AA': {'DD': 1, 'II': 1, 'BB': 1},
 'BB': {'CC': 1, 'AA': 1},
 'CC': {'DD': 1, 'BB': 1},
 'DD': {'CC': 1, 'AA': 1, 'EE': 1},
 'EE': {'FF': 1, 'DD': 1},
 'FF': {'EE': 1, 'GG': 1},
 'GG': {'FF': 1, 'HH': 1},
 'HH': {'GG': 1},
 'II': {'AA': 1, 'JJ': 1},
 'JJ': {'II': 1}}

In [3]:
test_flow_rates

{'AA': 0,
 'BB': 13,
 'CC': 2,
 'DD': 20,
 'EE': 3,
 'FF': 0,
 'GG': 0,
 'HH': 22,
 'II': 0,
 'JJ': 21}

## Part 1
---

What is the decision to make at each minute?

These are the state variables:
1. `current_valve` : Where we are in the graph
2. `time_left` : Countdown in minutes from 30
3. `open_valves` : An array keeping track of which of the `worthwhile_valves` are open (order matters)

At each minute, have to decide:
* Open current valve?
* Move to another valve?
* Stay put? (All `worthwhile_valves` are open already, so there's nothing worth doing)

While the test data is relatively small, the actual input data is significantly larger:

In [4]:
valve_graph, flow_rates = parse_input("valves")

print(f"There are {len(test_flow_rates)} in the *test* input data.")
print(f"There are {len(flow_rates)} in the *actual* input data.")

There are 10 in the *test* input data.
There are 60 in the *actual* input data.


But in each of the graphs, there are a *lot* of valves that do not work. That is, they have a `flow_rate` equal to `0`:

In [6]:
test_zero_flow_rates = [v for v in test_flow_rates if test_flow_rates[v] == 0]
zero_flow_rates = [v for v in flow_rates if flow_rates[v] == 0]

print(f"There are {len(test_zero_flow_rates)} valves in the *test* input data with flow_rate == 0.")
print(f"There are {len(zero_flow_rates)} valves in the *actual* input data with flow_rate == 0.")

There are 4 valves in the *test* input data with flow_rate == 0.
There are 45 valves in the *actual* input data with flow_rate == 0.


Going to a valve in the cave with with a flow rate of zero, isn't worth it.  
(Although, you may have to pass through such a room with a zero flow rate on your way to a room with a higher/positive flow rate.)

In [8]:
test_worthwhile_valves = [v for v in test_flow_rates if test_flow_rates[v] > 0]
worthwhile_valves = [v for v in flow_rates if flow_rates[v] > 0]

print (f"There are only {len(test_worthwhile_valves)} valves worth visiting in the *test* input data.")
print (f"There are only {len(worthwhile_valves)} valves worth visiting in the *actual* input data.")

There are only 6 valves worth visiting in the *test* input data.
There are only 15 valves worth visiting in the *actual* input data.


The test data have only 6 `worthwhile_valves`, but the actual input data have 15 `worthwhile_valves`.

While that doesn't *seem* like a big difference (6 vs. 15), it is!

That's because the number of possible permutations ***explodes*** as $n$ increases.  
The number of permutations of an array with $n$ elements is $n!$

In [8]:
import math
print(f" 6! is equal to: {math.factorial(6)}")
print(f"15! is equal to: {math.factorial(15)}")

 6! is equal to: 720
15! is equal to: 1307674368000


Despite this, collapsing the graph from 60 nodes down to just 15 will be beneficial.

I don't think there's a way to know this *a priori* but, the 30 min time constraint significantly reduces the **1.3 trillion** possible permutations/paths to explore to find the optimal (max) pressure release.

To collapse the graph, we need an algorithm to find the distance between every worthwhile valve.

The [Floyd-Warshall](https://en.wikipedia.org/wiki/Floyd%E2%80%93Warshall_algorithm) algorithm does just that.  
That algorithm finds the minimum distance between *all* nodes in a graph. So, that is, all 60 nodes.  
In many implementations, it returns an $n$ by $n$ matrix of all the distances between nodes.

This is a bit overkill, algorithmically.  
We only need a $15$ by $15$ matrix, but I'm not going to worry about it.

Floyd-Warshall time complexity is $O(n^3)$ and 60 nodes is still small enough for it to solve relatively quickly.

An alternative, and likely faster, solution would be to do a breadth-first search (BFS) for the $15$ by $15$ matrix.

In [11]:
from collections import defaultdict
from itertools import product

def fw(graph):
    """See algorithms/floyd_warshall.ipynb for comments/details."""
    dist = defaultdict(lambda: float("inf"))

    for node, neighbors in graph.items():
        dist[(node, node)] = 0
        for neighbor, edge_weight in neighbors.items():
            dist[(node, neighbor)] = edge_weight

    for k, i, j in product(graph.keys(), repeat=3):
        dist[(i, j)] = min(dist[(i, j)], dist[(i, k)] + dist[(k, j)])

    return dict(dist)

(For no real good reason, I'm creating a dictionary with $n * n$ entries instead of a $n$ x $n$ matrix.)

In [12]:
test_min_dists = fw(test_valves_graph)
dict(list(test_min_dists.items())[:5])

{('AA', 'AA'): 0,
 ('AA', 'DD'): 1,
 ('AA', 'II'): 1,
 ('AA', 'BB'): 1,
 ('BB', 'BB'): 0}

Next, we need an algorithm to explore all the possible paths and keep track of the maximum release possible for each path.

I played ***A LOT*** with different implemenations of the below, trying to find some improvements with just returning the max value and implementing some memoization, but none of them seemed to actually improve the run time of the solution. (In fact, in several instances, the overhead of adding a `functools.cache` decorator seemed to *slow* things down rather than speed it up. Head scratching.)

I also liked exploring all the possible paths this way because I could debug by saving both the actual path and the max release and test that everything was working as expected when I limited the number of `worthwhile_valves` to just 2 or 3 potentials before the computation explodes.

In [31]:
def calc_all_possible_releases(
        curr_valve, 
        path_so_far="", 
        time_left=30, 
        release=0,
        min_dists=None,
        flow_rates=None,
        worthwhile_valves=None
    ):

    all_possible_releases = []
    
    # path_so_far is stored as a string, so it's immutable, and thus hashable,
    # whereas a list would not be. The + "," becomes important in the line below:
    # if valve not in path_so_far because in the actual data the valve labels 
    # are not all double characters and so some nodes match with the last/first
    # characters of previous/current node labels in the path.
    path_so_far += curr_valve + "," 

    if len(path_so_far) > 3:
        prev_valve = path_so_far[-6:-4]
        travel_time = min_dists[(prev_valve, curr_valve)]
        time_left -= (travel_time + 1)  # + 1 for time it takes opening valve

    # Base case
    if time_left <= 0:
        return all_possible_releases
    
    else:
        release += time_left * flow_rates[curr_valve]
        all_possible_releases += [release]

        for valve in worthwhile_valves:
            if valve not in path_so_far:
                # Recursive call
                all_possible_releases += calc_all_possible_releases(
                    valve,
                    path_so_far,
                    time_left, 
                    release, 
                    min_dists,
                    flow_rates,
                    worthwhile_valves
                )
            
        return all_possible_releases

In [26]:
def solve_part1(filename):
    valves_graph, flow_rates = parse_input(filename)

    # Only want to consider valves that have positive flow rates
    worthwhile_valves = [v for v in flow_rates if flow_rates[v] > 0]
    min_dists = fw(valves_graph)

    all_possible_releases = calc_all_possible_releases(
        "AA", 
        min_dists=min_dists,
        flow_rates=flow_rates, 
        worthwhile_valves=worthwhile_valves
    )

    return max(all_possible_releases), min(all_possible_releases), len(all_possible_releases)

### Run on Test Data

In [34]:
solve_part1("test_valves")[0] == 1651

True

### Run on Input Data

In [35]:
solve_part1("valves")

(2265, 0, 136465)

## Part 2
---

I can't even think about looking at this right now...

### Run on Test Data

### Run on Input Data